
![Youtube](https://upload.wikimedia.org/wikipedia/commons/e/e1/Logo_of_YouTube_%282015-2017%29.svg)
# Title Generator

#### Imports

In [13]:
import jsonpath_ng
import requests
import json
from Utils.TextUtils import clean_txt, generate_title
from Utils.Markov import make_markov_model
from Utils.dict_utils import summarize_inner_dict
import random
import pandas as pd
import regex as re
import ProtoClasses.trending_pb2 as trending_pb2
import ProtoClasses.trending_pb2_grpc as trending_pb2_grpc
import ProtoClasses.video_pb2 as video_pb2
import ProtoClasses.video_pb2_grpc as video_pb2_grpc
import grpc

#### Requests
Make the request to the webapi to get the data from the trending pages of the wanted countries

In [5]:
countries = ['US','GB', 'AU', 'CA']
jsonList = []
for country in countries:
    response = requests.get(f"https://localhost:7244/api/trending/{country}/true", verify=False)
    tempList = json.loads(response.content)
    tempList = [summarize_inner_dict(dictionary, 'uploader', 'name') for dictionary in tempList]
    jsonList.extend(tempList)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py:1043: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warn

In [16]:
channel = grpc.insecure_channel("localhost:5117")
stub2 = trending_pb2_grpc.TrendingStub(channel)
response2 = stub2.GetMultipleTrendingVideos(trending_pb2.MultipleTrendingRequest(TrendingRequest =
[trending_pb2.TrendingRequest(trendingId = 'US'), trending_pb2.TrendingRequest(trendingId = 'CA')]))
response2



VideoData {
  videoId: "m182yfkVqP8"
  uploader {
    name: "euphoria"
    subscribers: "735\302\240mil subscritores"
  }
  length: "00:01:07"
  country: "US"
  uploadDate: "20/02/2022 00:00:00"
  isFamilyFriendly: true
  iFrame: "https://www.youtube.com/embed/m182yfkVqP8"
  category: "People & Blogs"
  views: 6849745
  likes: "190\302\240988 gostos"
  title: "euphoria | season 2 episode 8 promo | hbo"
  description: "on the season finale of #EUPHORIA\r\n\r\nZendaya leads an ensemble cast that also includes Maude Apatow, Angus Cloud, Eric Dane, Alexa Demie, Jacob Elordi, Barbie Ferreira, Nika King, Storm Reid, Hunter Schafer, Algee Smith and Sydney Sweeney. \r\n\r\nAll episodes created and written by Sam Levinson, who also serves as executive producer. Other executive producers include Drake, Future the Prince, Ravi Nandan, Kevin Turen, Hadas Mozes Lichtenstein, Ron Leshem, Daphna Levin, Tmira Yardeni, Mirit Toovi, Yoram Mokady, Gary Lennon and Jim Kleverweis. Produced in partnership w

#### Exploring datataset

In [6]:
youtube_data = pd.DataFrame(jsonList)
youtube_data.drop_duplicates(keep='first', inplace=True, subset='videoId')
youtube_data

,videoId,uploader,length,country,uploadDate,isFamilyFriendly,iFrame,category,views,likes,title,description
0,WieBHoGRsoU,DaBaby,00:04:11,US,2022-03-02T00:00:00,True,https://www.youtube.com/embed/WieBHoGRsoU,Music,2170758,171 026 gostos,DaBaby X NBA YoungBoy - NEIGHBORHOOD SUPERSTAR...,"""BETTER THAN YOU"" MARCH 4TH\n\nPRE SAVE NOW : ..."
1,MVu8QbxafJE,Vox,00:08:49,US,2022-03-02T00:00:00,True,https://www.youtube.com/embed/MVu8QbxafJE,News & Politics,4027134,117 117 gostos,"Putin's war on Ukraine, explained",Ukraine is under attack. Follow Vox for the la...
2,nrWIdnJO15c,RAPSTAR,00:02:47,US,2022-03-02T00:00:00,True,https://www.youtube.com/embed/nrWIdnJO15c,Music,383329,22 710 gostos,Boosie Badazz - ''I Don't Call Phones I Call S...,Boosie Badazz - ''I Don't Call Phones I Call S...
3,gSvc9aBu4O0,Elley Duhé,00:03:08,US,2022-01-03T00:00:00,True,https://www.youtube.com/embed/gSvc9aBu4O0,Entertainment,849507,,Elley Duhé - MIDDLE OF THE NIGHT (Official Video),Listen to MIDDLE OF THE NIGHT\nhttps://smartur...
4,5Gekes-KoSc,Les Do Makeup,00:11:48,US,2022-03-02T00:00:00,True,https://www.youtube.com/embed/5Gekes-KoSc,Howto & Style,821651,79 357 gostos,THE PROPOSAL!! *EMOTIONAL*,Hi babygirls! Thank you so much for watching a...
...,...,...,...,...,...,...,...,...,...,...,...,...
325,0uav6zQKiMQ,IGN,00:12:01,CA,2022-02-23T00:00:00,True,https://www.youtube.com/embed/0uav6zQKiMQ,Gaming,3060337,76 708 gostos,Elden Ring Review,Watch our Elden Ring review. Elden Ring is a m...
327,ziRI-mJPI_U,TODAY,00:01:46,CA,2022-02-24T00:00:00,True,https://www.youtube.com/embed/ziRI-mJPI_U,News & Politics,2041269,21 910 gostos,How The Russian Public Feels About The Ukraine...,"Reporting for TODAY from Moscow, NBC’s Kier Si..."
329,tx9IsK8qhg4,Boomstick Gaming,00:11:21,CA,2022-02-24T00:00:00,True,https://www.youtube.com/embed/tx9IsK8qhg4,Gaming,2436376,88 172 gostos,ELDEN RING | Get “Overpowered” At The Very Start,"Elden Ring gameplay guide, with tips for “over..."
332,sRp_xrEt_wc,Feelgoodfoodie,00:00:38,CA,2022-02-20T00:00:00,True,https://www.youtube.com/embed/sRp_xrEt_wc,People & Blogs,7411797,411 697 gostos,Have you seen this crazy sink?! | FeelGoodFoodie,


In [7]:
def clean_likes(likes):
    if  likes:
        likes = likes.replace('gostos', '').replace(" ", '')
        likes = re.sub(r'[^\x00-\x7F]+','', likes)
    elif not likes:
        likes = 0
    return likes
youtube_data = youtube_data.astype("string")

youtube_data['likes'] = youtube_data['likes'].apply(clean_likes)
youtube_data['likes'] = youtube_data['likes'].astype('int64')
youtube_data['views'] = youtube_data['views'].astype('int64')
youtube_data['isFamilyFriendly'] = youtube_data['isFamilyFriendly'].astype('bool')

In [8]:
youtube_data.category.value_counts()

Entertainment           38
Gaming                  28
People & Blogs          25
Music                   24
News & Politics         21
Sports                  17
Comedy                  10
Education                8
Howto & Style            7
Science & Technology     3
Film & Animation         2
Autos & Vehicles         1
Travel & Events          1
Pets & Animals           1
Name: category, dtype: Int64

In [9]:
titles = youtube_data['title'].to_list()
cleaned_titles = clean_txt(titles)

In [11]:
markov_model = make_markov_model(cleaned_titles, 1)
newtitles = []
for _ in range(20):
    title = generate_title(markov_model, start=random.choice(cleaned_titles), limit=random.randint(3,11))
    newtitles.append(title)
    print(title)

of the last to opening night primitive technology thatched 
hippie shorts throwing school busses with the biggest thing weve 
for more viral bmx getting braces shorts bass drop 
paul giant coral trout catch and it out official trailer daz watches 
shots youngboy unreleased live live storm eunice at un security council 
viral los gemelos de fc 
racing war crimes in 
never go on ukraine watch till the night official 
v chelsea v tottenham key moments fifth round emirates fa cup final 
catch and gottmik rem beauty how i built 
movie shorts facelifting hours with the end it s 
biden says putin will delete later what the end 
quête clip officiel twins around the world record liverpool carabao cup final 
with a character too strong sifu got me 
goes sneaker shopping with my tourette s 
difficulty mode emotional damage finding new daily routine russia 
frank smith in a trampoline park 
time to shine nba youngboy unreleased live live tyson fury 
on a wireless hotspot that large explosions repo